In [5]:
import torch
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import models


In [6]:
def CIFAR10_iid(datapoints, path):


    #these transforms are same as Imagenet configurations as TL is being used
    custom_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
    ])
    train_dataset = torchvision.datasets.CIFAR10(root=path,
                                            train=True,
                                            transform=custom_transform,download= True)

    test_dataset = torchvision.datasets.CIFAR10(root=path,
                                            train=False,
                                            transform=custom_transform, download = True)
    
    class2idx = train_dataset.class_to_idx.items()
    idx2class = {v: k for k, v in train_dataset.class_to_idx.items()}
    
    new_train_dataset_size =  datapoints
    temp = len(train_dataset) - new_train_dataset_size

    print(len(train_dataset), new_train_dataset_size, temp)

    new_train_dataset,_ = torch.utils.data.random_split(train_dataset, (new_train_dataset_size, temp))
    new_test_dataset,_ = torch.utils.data.random_split(test_dataset, (2000, 8000))  #keeping 2k datapoints with each client

    return new_train_dataset, new_test_dataset, dict(class2idx), idx2class


In [7]:
train_dataset, test_dataset = CIFAR10_iid(500, "data")

NameError: name 'torchvision' is not defined

In [3]:
def create_DataLoader(train_dataset, test_dataset, train_batch_size, test_batch_size):
    train_batch_size = train_batch_size
    test_batch_size = test_batch_size
    train_DataLoader = torch.utils.data.DataLoader(dataset= train_dataset,
                                            batch_size=train_batch_size,
                                            shuffle=True)
    test_DataLoader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=test_batch_size,
                                            shuffle=True)

    return train_DataLoader, test_DataLoader

In [17]:
model = models.resnet18(weights="IMAGENET1K_V1")


In [18]:
# for param in model.parameters():
#     param.requires_grad = False

In [19]:
model.fc = nn.Linear(512,10,bias = True)

In [13]:
for param in model.layer4.parameters():
    param.requires_grad = True
    
# for param in model.layer3.parameters():
#     param.requires_grad = True

In [20]:
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def compute_accuracy(model, data_loader):
    model.eval()
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)

        logits = model(features)
        _, predicted_labels = torch.max(logits, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100


def compute_epoch_loss(model, data_loader):
    model.eval()
    curr_loss, num_examples = 0., 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.to(DEVICE)
            targets = targets.to(DEVICE)
            logits = model(features)
            loss = F.cross_entropy(logits, targets, reduction='sum')
            num_examples += targets.size(0)
            curr_loss += loss

        curr_loss = curr_loss / num_examples
        return curr_loss
    
    

start_time = time.time()
for epoch in range(num_epochs+1):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
    print(f"Epoch {epoch} completed!")
    
    if epoch%2 == 0:
        model.eval()
        with torch.set_grad_enabled(False): # save memory during inference
            print('Epoch: %03d/%03d | Train: %.3f%% | Loss: %.3f' % (
                  epoch, num_epochs+1, 
                  compute_accuracy(model, train_loader),
                  compute_epoch_loss(model, train_loader)))

        with torch.set_grad_enabled(False): # save memory during inference
            print('Epoch: %03d/%03d | Test: %.3f%%' % (
                  epoch, num_epochs+1, 
                  compute_accuracy(model, test_loader)))
        

    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch 0 completed!
Epoch: 000/011 | Train: 75.450% | Loss: 0.716
Epoch: 000/011 | Test: 71.670%
Time elapsed: 0.48 min
Epoch 1 completed!
Time elapsed: 0.78 min
Epoch 2 completed!
Epoch: 002/011 | Train: 86.280% | Loss: 0.399
Epoch: 002/011 | Test: 77.930%
Time elapsed: 1.24 min
Epoch 3 completed!
Time elapsed: 1.54 min
Epoch 4 completed!
Epoch: 004/011 | Train: 91.742% | Loss: 0.241
Epoch: 004/011 | Test: 79.940%
Time elapsed: 2.02 min
Epoch 5 completed!
Time elapsed: 2.32 min
Epoch 6 completed!
Epoch: 006/011 | Train: 96.294% | Loss: 0.116
Epoch: 006/011 | Test: 80.900%
Time elapsed: 2.77 min
Epoch 7 completed!
Time elapsed: 3.08 min
Epoch 8 completed!
Epoch: 008/011 | Train: 97.194% | Loss: 0.084
Epoch: 008/011 | Test: 80.460%
Time elapsed: 3.53 min
Epoch 9 completed!
Time elapsed: 3.84 min
Epoch 10 completed!
